In [ ]:
# All this notebook has been used on  google collab platform and NOT localy using T4 GPU

!pip install fastapi 
!pip install colabcode
!pip install uvicorn==0.15.0
!pip install pyngrok==5.0.0

!ngrok authtoken 2VxpDsoeuVw9C5BCDjoZeiH89Iq_2TGwbQGBKYYu7oo26PAwv
!ngrok http -host-header=rewrite localhost:PORT

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.*")
import os

from torch.optim.lr_scheduler import StepLR
import torch.optim as optim

import string
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoModelForSequenceClassification
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup

import multiprocessing

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau , CosineAnnealingLR
from sklearn.metrics import f1_score, jaccard_score
import ast
import numpy as np
from scipy import stats
import math
from sklearn import datasets
from sklearn import metrics
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import copy

import random

plt.style.use('ggplot')


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re


def preprocessed_text(text):
    # Define the regular expression pattern for French punctuation marks
    punctuation_pattern = r'[,?!;:…()\[\]«»—/""\'\']'
    
    # Remove HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text()
    
    # Use re.sub() to remove the matched punctuation marks
    cleaned_text = re.sub(punctuation_pattern, '', text_without_tags)
    
    # Tokenization
    tokens = nltk.word_tokenize(cleaned_text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
              if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [ ]:
def predict_tags(model, input_question, threshold=0.2):
    input_question.lower()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    input_ids = tokenizer.encode(input_question, max_length=tokenizer.model_max_length, truncation=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(input_ids)
    
    predicted_probabilities = torch.nn.functional.sigmoid(outputs.logits)   # Sigmoid because  it is a  multilabel text classification,and softmax if it is a  mutli class task
    tag_probabilities = predicted_probabilities.squeeze().tolist()
    
    total_probability = round(sum(tag_probabilities) , 6)

    tag_labels = y.columns  

    decimal_places = 3
    
    tag_predictions = [{"tag": tag_labels[i], "probability": prob} for i, prob in enumerate(tag_probabilities) if prob > threshold]
    
    sorted_tag_predictions = sorted(tag_predictions, key=lambda x: x["probability"], reverse=True)

    print(f"Here is the sum of the total probability of all 60 tags :  {total_probability} " )

    # Print the sorted tag predictions above the threshold
    for tag_prediction in sorted_tag_predictions:
        rounded_probability = round(tag_prediction['probability'], decimal_places)
        print(f"Tag: {tag_prediction['tag']}, Probability: {rounded_probability}")
    
    #returning the suggested tags in an array format
    return [item['tag'] for item in sorted_tag_predictions]

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

loaded_model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=labels_number
)

# Load the saved model state dictionary
model_path = "/content/drive/My Drive/bert_model_2e-5.pth"

loaded_model.load_state_dict(torch.load(model_path))

In [ ]:
from fastapi import FastAPI, HTTPException , Query
from pydantic import BaseModel
import torch
from pyngrok import ngrok
import nest_asyncio
from colabcode import ColabCode


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

loaded_model.eval()

app = FastAPI()


class QuestionInput(BaseModel):
    question: str

@app.get("/predict/")
async def predicting_tags(question: str = Query(..., description="Input question for prediction")):
    # Preprocess the input question
    preprocessed_question = preprocessed_text(question)

    # Predict tags
    tags = predict_tags(loaded_model, preprocessed_question , threshold=0.1)
    
    # Create a response string that combines the question and tags
    question_text = f"Input Question: {question}"
    tags_text = f"Predicted Tags: {', '.join(tags)}"

   # Combine response_text and tags_text with a newline character
    combined_response = question_text + " |¤| " + tags_text

    # Return the response text as plain text
    return combined_response


In [ ]:
# Start the FastAPI app using ColabCode
cc = ColabCode(port=8000, code=False)
cc.run_app(app=app)


# Add "/predict/?question=How can I input an image into my website ?" to the ngrok tunnel URL to input a question for example